# A Valuation of Public Parks

Authors: Chad Atalla, Alicia Chen, Nadah Feteih, Alan Chen, Joshua Van Gogh, Anjali Verma

## Introduction and Background
----
What value does establishing public parks add to a city and a community? Parks provide societal, commercial, and environmental benefits to the surrounding regions that are often overlooked. We will utilize hedonic pricing method to explore the value of parks and where they will provide the most value by using publicly available data from San Diego county. 
 
As we begin our research, we hypothesize that the true monetary value of public parks greatly exceeds the actual cost of land allocation and construction. This value will vary based on location and park "features" (house prices, health costs, etc.), but data driven estimates of this value will make it easier for cities to justify park construction costs and choose optimal locations.

We have decided to collect and analyze data from these different features and categorize their impact based on police beat. Our goal is to find whether certain features strongly indicate an overall benefit in establishing public parks and if there is a strong correlation between these features and the existence of public parks in these areas.   

## Data Description 
----

We have decided to collect data from various sources in order to represent these features. These datasets include information about existing park locations, community recreational courses, environmental factors (storm drains), health (cost of treatment), housing (market prices), and police activity. 

**Dataset Name**: Park Locations (San Diego)
* **Link to the dataset**: https://data.sandiego.gov/datasets/park-locations/
* **Number of observations**: 258
* **Description**: This dataset contains park locations, size, and basic content features (playgrounds, etc). The dataset is a SHP file, (common GIS format).


**Dataset Name**: Zillow API
* **Link to the dataset**: https://www.zillow.com/howto/api/APIOverview.htm
* **Number of observations**: Number of observations is dependent on areas included in investigation but in San Diego County. We were able to extract around 330 observations to represent locations all around San Diego. 
* **Description**: This dataset contains “Zestimates” of the monetary values of properties in specified areas. It also contains many details about all of the properties themselves.

**Dataset Name**: Health Benefits datasets
* **Link to the dataset**: http://www.sandiegocounty.gov/content/dam/sdc/hhsa/programs/phs/documents/CHS-HealthStatusReport_Central_2012.pdf  
  http://www.sandiegocounty.gov/content/dam/sdc/hhsa/programs/phs/documents/CHS-HealthStatusReport_NorthCentral_2012.pdf
* **Number of observations**: Aggregate data for the regions of San Diego County
* **Description**: The report contains community health statistics for diabetes, obesity, cancer, etc.

**Dataset Name**: Environmental Benefits
* **Links to the datasets**: http://rdw.sandag.org/Account/gisdtview?dir=StormDrain
http://www.sciencedirect.com/science/article/pii/S1618866715300911	 	
* **Number of observations**: 55,187 storm drains, Economic cost per ton of CO2, Average CO2 sequestered per hectare by parks
* **Description**: We expect parks to provide many environmental benefits including a reduction in storm runoff and CO2 sequestering. We equate the necessity of storm drains to the amount of storm runoff a region has. Parks remove an estimated cost of 23,000 dollars worth of CO2 per ha per year. We estimated that the parks in San Diego did not have as much shrubbery as the parks in the study so we rounded that value down to a conservative 1,000 dollars per acre per year.


**Dataset Name**: Recreational Activity
* **Link to the dataset**:  https://apm.activecommunities.com/sdparkandrec/Activity_Search?ActivityCategoryID=1&isSearch=true&applyFiltersDefaultValue=true
* **Number of observations**: ~1,600 
* **Description**: The community activity dataset comes from The City of San Diego Parks and Recreation Department. It details information about different general recreation and sports programs available in parks across San Diego County. The website contains lots of information about each program but for our purposes we only focused on locations and cost per individual to participate.

**Dataset Name** : Safety / police activity
* **Links** : https://data.sandiego.gov/datasets/police-beats/ - beat names and numbers
https://data.sandiego.gov/datasets/police-calls-for-service/ - calls made to dispatch
* **Number of observations**: ~586,000 dispatches in 2016, 128 police beats
* **Description**: The first link provides a shapefile and csv file of the police beats of San Diego. The shapefile is used for all of our map drawings and heatmaps. The second link provides csv files for the police calls for service, we use data from 2016 and take the columns referring to dispatch instance and beat.


## Data Cleaning
----

Each individual source of data had to be cleaned and sanitized in a different way in order to standardize the dataset. We have each occurrence of each dataset mapped to some police beat and its corresponding beat number, as well the monetary value we have assigned to it. 

In [ ]:
!pip install shapely
!pip install pyshp
!pip install geopy
!pip install python-zillow

In [ ]:
import numpy as np
import pandas as pd
import shapefile
import zillow
import random
import json
import sys,argparse,csv
from geopy.geocoders import Nominatim
from beat_from_coord import *

### Park Data
The data comes in a SHP file but all we needed to know was the number and total areas of the parks within each beat. There were also a few cemeteries included in the dataset so we removed those as well since they aren’t what our hypothesis is about.

In [ ]:
sf = shapefile.Reader("city_parks/CITY_PARKS")

# get field names and data
fields = sf.fields[1:]
attributes = sf.records()

# gather names of columns to a list
column_names = [i[0] for i in fields]

# create dataframe from the attributes
df = pd.DataFrame(attributes)

# rename column names from indices to their column name
df =df.rename(columns={v: k for v, k in enumerate(column_names)})

# add beats to dataframe
for index, row in df.iterrows():
    lat = float(df.iloc[index]['Y_COORD'])
    lon = float(df.iloc[index]['X_COORD'])
    
    if (pd.notnull(lat) and pd.notnull(lon)):
        beat = beat_from_coord(lat, lon)
        df.set_value(index, 'BEAT_NUM', beat[0])
        df.set_value(index, 'BEAT', beat[1])
        
# reduce columns
df = df[['OBJECTID','COMMON_NAM','DESIG_USE','ACRES','DEDIC_PARK','BEAT','BEAT_NUM']]

# remove cemeteries and failed beats
df = df[df['DESIG_USE'] != 'Cemetery']
df = df[df['BEAT_NUM'] != -1]

# create dataframe that contains the beat_num, number of parks in the beat, and total park area.
parks_in_beat = df['BEAT_NUM'].value_counts()
beat_df = df.groupby('BEAT_NUM').aggregate(sum)
beat_df['NUM_OF_PARKS'] = parks_in_beat
beat_df['BEAT_NUM'] = beat_df.index

# Get the beat area names
sf = shapefile.Reader("data/beats/SDPD_BEATS")
shapes = sf.shapes()
records = sf.records()
beat_names = pd.read_csv('data/beats/beat_names.csv')
beat_names['Neighborhood'] = beat_names['Neighborhood'].str.lower().str.strip()

# Add the total beat area to beat df
for record in records:
    name = record[0]
    name = record[0].lower().strip()
    name = name[:name.find('\x00')]
    name = exception_lookup(name)
    beat = beat_names[beat_names['Neighborhood'] == name]['Beat'].tolist() 
    beat = beat[0] if beat != [] else -1
    
    if beat != -1:
        index = beat_df[beat_df['BEAT_NUM'] == beat].index
        beat_df.set_value(index, 'BEAT_AREA', record[1])

# Write gathered data to csv
beat_df.to_csv('data/parks/beat_to_parks.csv')
beat_df

### Housing Prices

We are valuing houses in a region based on the average market price. In order to collect data on the market prices for houses in each police beats, we are using the Zillow API to extract the "Zestimate". We are randomly sampling data from the houses listed on Zillow in San Diego since we arent able to pull large amounts of data from specific areas. 

In [ ]:
if __name__=="__main__":
    key = "X1-ZWz196b0emyd57_8p8sy"

    f = open('houseData.json', 'w')
    minL = 16717995 
    maxL = 16834401
    list = []
    for x in range(100):
        r = random.randint(minL,maxL)
        list.append(r)

    for y in range(100):
        api = zillow.ValuationApi()

        detail_data = api.GetZEstimate(key, list[y])
        f.write(json.dumps(detail_data.get_dict()))

We are extracting the latitute and longitude and placing these houses in different police beats with the value being the Zestimate (market price) of the house. We are cleaning the data by removing any houses that dont correspond to any beats. 

In [ ]:
info = []
houses = pd.read_json('houseData.json', lines=True)
#houses = houses.head(n=5)
for i, row in houses.iterrows():
    lat, long, am = row['full_address']['latitude'], row['full_address']['longitude'], row['zestimate']['amount']
    beat, name = beat_from_coord(float(lat), float(long))
    if (beat > -1): info.append((beat, name, am))
    
with open('houseVals.csv','w') as out:
    csv_out=csv.writer(out)
    csv_out.writerows(info)

### Environmental Factors
We obtained a SHP file of the locations of all the storm drains in San Diego from the Sandag website. The locations were all in the X and Y geographical coordinate system, so we imported the SHP file and DBF file into mapshaper, which we used to convert the coordinates to longitude and latitude. Using mapshaper, we were also able to convert the SHP file into a CSV file. We were then able to use our beat_from_coord function to map the longitude and latitude coordinates of each storm drain to a police beat. Using the police beat locations of each storm drain, we counted the total number of storm drains in each beat and multiplied that number by the average storm drain cost that we obtained from Homeadvisor. This allowed us to assign a monetary value to each beat area based on number of storm drains in the beat area.

In [ ]:
#convert the csv file of storm drain beat areas into a dataframe
df = pd.read_csv('st_beats.csv')
#remove data points that have an unknown beat number
df = df[df.Beat != -1]
#get number of storm drains in each beat area
df_counts = df['Beat'].value_counts()

#read in the csv file of beat number and beat names
myDict = {}
f=open('beat_names.csv', 'r')
reader=csv.reader(f)
#map beat number to beat name in a dictionary
for line in reader:
    if(line[0].isdigit()):
        myDict[line[0]]=line[1]
f.close()

#create a list of beat names in the order that the beat numbers appear 
list_names = []
for entry in df_counts.index:
    list_names.append(myDict[str(entry)])

#create a list of the moneatary values of each beat area
myList = []
for i in df_counts.values:
    myList.append(i*(-3371))

#create a new dataframe with the beat numbers corresponding to value
new_df = pd.DataFrame({'Beat':df_counts.index, 'Count':df_counts.values, 'Value':myList, 'Name':list_names})
print(new_df)
#write the dataframe to a csv file
new_df.to_csv('env_value.csv',sep=',')


### Police Activity 
Since we settled on SD police beats as our regional groupings, cleaning the dispatch data was relatively simple. The dataset was made available on the San Diego data portal, and was already relatively clean: including a dispatch time and beat on each line. We removed all NaN beat entries and dropped all other columns, then grouped and counted by beat. This gave us the resulting csv with police dispatch instances per beat in the year 2016.

In [ ]:
DOLLAR_PER_DISPATCH = 100

df = pd.read_csv('data/police_dispatch/police_dispatch_2016.csv')
bt = pd.to_numeric(df['beat'], errors='coerce')
bt = bt.dropna()
bt = bt.astype(int)
btv = bt.value_counts()

rbts = pd.read_csv('data/beats/beat_names.csv')
rbtList = rbts['Beat'].astype(int).tolist()

btv = btv.to_frame()
btv['beat_n'] = btv.index
btv['dollars'] = btv['beat']*DOLLAR_PER_DISPATCH
btv['valid'] = btv['beat_n'].map(lambda x: x in rbtList) 
btv = btv[btv.valid == True]

brief = btv[['dollars','beat_n']]
brief = brief.reset_index()
brief.to_csv('data/police_dispatch/police_dispatch_dollars_beat.csv')

### Health Care Costs

The data comes from tables in PDF format with health data from the central region of San Diego. We researched to find dollar values for treating four different health conditions and based on the number of people in each region found in the tables, assigned the dollar values to the corresponding regions. 

In [ ]:
df_health = pd.read_csv('data/health/healthDataUpdated.csv')
location = df_health.iloc[0]['Region']
lat = float(location[:-14])
lon = float(location.split(",",1)[1])

for index, row in df_health.iterrows():
    location = df_health.iloc[index]['Region']
    lat = float(location[:-13])
    lon = float(location.split(",",1)[1])
    #df_health[index]['Region'] = beat_from_coord(lat, lon)
    df_health.set_value(index, 'Region', beat_from_coord(lat, lon))


### Recreational Activity Data Cleaning
The data was scraped from the website using the ParseHub app. We pulled program name, program cost per individual, and program location. After cleaning we wanted the data to map beats to a monetary value based on the dataset. We first extracted the latitudes and longitudes of each program using the address and a python library geopy. We also wanted to make sure to only use the programs that had location values.

In [ ]:
df_activities = pd.read_csv('data/community/Activities.csv')

#fix the one funky data entry... this is janky but it works
df_activities['Activity_Location'][69] = "8285 Skyline Dr\nSan Diego, CA, US 92114"

#drop activities that don't have a location
df_activities = df_activities.dropna()
df_activities = df_activities.reset_index(drop=True)

The addresses were in a format that the geopy library couldn't use. So we created a helper function to clean up the address formats.

In [ ]:
def clean_address(addr):
    addr = addr.replace("\n", " ", 1)
    addr = addr[:-14]
    return addr

Now we can pull the latitude and longitudes! 

In [ ]:
geolocator = Nominatim()

latitude = []
longitude = []

for address in df_activities['Activity_Location']:
    clean = clean_address(address)
    geo = geolocator.geocode(str(clean))
    if geo is None:
        lat = np.nan
        long = np.nan
    else:
        lat = geo.latitude
        long = geo.longitude
    latitude.append(lat)
    longitude.append(long)
    
df_activities['Latitude'] = latitude
df_activities['Longitude'] = longitude

# get rid of activities that we couldn't find a longitude and latitude for
df_activities = df_activities.dropna()
df_activities = df_activities.reset_index(drop=True)

Then using the function beat_from_coord  we mapped each latitude and longitude to a police beat. The website did not provide how many participants were in each program so we used Fermi’s approximation to estimate the monetary value of each program. Thus, we multiplied the program cost per individual by 10 to get the monetary value each program added. 

In [ ]:
beats = []
for index, row in df_activities.iterrows():
    #extract the beat from the lat and long
    lat = float(df_activities.iloc[index]['Latitude'])
    lon = float(df_activities.iloc[index]['Longitude'])
    beats.append(beat_from_coord(lat, lon))
    
    price_str = df_activities.iloc[index]['Activity_price']
    if ( price_str.find("Free") == 0 ):
        df_activities.set_value(index, 'Activity_price', 0)
    else:
        #Fermi's approximation
        df_activities.set_value(index, 'Activity_price', float(price[1:-18])*10)
    
df_activities['Beat'] = beats

#don't need these columns anymore
df_beat_money = df_activities.drop(['Activity_url', 'Activity_Location','Activity_name', 'Clean_Addresses', 'Lattitude', 'Longitude'], axis=1)


## Data Visualization and Analysis
----

After initially cleaning each dataset, we decided to visualize the monetary value that each of these features adds to a region by graphing the correlation of each observation with amount of park space. 

In [ ]:
# Additional libraries used in visualizing data
%matplotlib inline

import matplotlib.pyplot as plt
import patsy
import statsmodels.api as sm
#from scipy.stats import ttest_ind
from scipy import stats

Some of our data had major outliers that affected our predictions. This function removes them from the df

In [ ]:
# Make outlier remover tool
def rem_outlier(df_in, col_name, count, how=['max', 'min']):
    fn = max if how == 'max' else min
    rem = df_in[df_in[col_name] == fn(df_in[col_name])]['Beat']
    df_no = df_in[df_in.Beat != rem.values[0]]
    for x in range(0,count-1):
        rem = df_no[df_no[col_name] == fn(df_no[col_name])]['Beat']
        df_no = df_no[df_no.Beat != rem.values[0]]
    return df_no

During our visualization we found out that we needed to clean our data a bit more due to outliers and some incorrect data. The most notable changes include the removal of Balboa Park, as it is more a super park/cultural center than the community parks we are trying to analyze, and the reduction of Mission Bay's park area so it would only include its land mass.

In [ ]:
# Pair number of parks and monetary value based on beat for each factor,  then find correlation
df_parks = pd.read_csv('data/parks/beat_to_parks.csv')
df_beats = pd.read_csv('data/beat_names_areas.csv')

df_beats = df_beats.dropna()

df_beats.drop('Neighborhood', axis=1, inplace=True)
df_parks.drop('OBJECTID', axis = 1, inplace=True)
df_parks.drop('BEAT_NUM.1', axis = 1, inplace = True)
df_parks.drop('BEAT_AREA', axis=1, inplace=True)

df_parks=df_parks.rename(columns = {'BEAT_NUM':'Beat'})
df_parks=df_parks.rename(columns = {'ACRES':'PARK_ACRES'})

# Correct for land area of mission bay
for i in range(0, len(df_parks)):
    if(df_parks.ix[i, 'Beat'] == 121):
        df_parks.ix[i, 'PARK_ACRES'] = 83.16
        
df_parks = pd.merge(df_beats, df_parks, how='left')
df_parks=df_parks.rename(columns={'Area':'BEAT_AREA'})
df_parks = df_parks.fillna(value=0)

# Remove outliers
df_parks = rem_outlier(df_parks, 'PARK_ACRES', 3, how='max')

Before we start analyzing any data, we can visualize where parks are in San Diego.

**Number of Parks per Beat**
<img src="Pr_045-master/num_parks.png">

**Acres of Park Land per Beat**
<img src="Pr_045-master/acres.png">

### Health
For health costs we first negated and then normalized all of our values for health cost by the beat size. This is because the larger the beat is the more people it is likely to have. Unfortunately, we did not have the population for beat so we had to make do with size. There is a fairly positive correlation value, showing that more parks correlate to a smaller ratio of health costs/area in a beat.

In [ ]:
df_combined = pd.merge(df_parks, df_health, on='Beat', how='left')
df_combined = df_combined.fillna(value=0)

# normalize value/beat_area --> $ per foot
for i in range(0, len(df_combined)):
    if (df_combined.ix[i, 'BEAT_AREA'] != 0):
        df_combined.ix[i, 'Value'] =  df_combined.ix[i, 'Value']/ df_combined.ix[i, 'BEAT_AREA']

# Remove outliers
df_combined = rem_outlier(df_combined, 'Value', 4, how='min')
        
health_r_n, health_p =stats.pearsonr(df_combined['NUM_OF_PARKS'], df_combined['Value'])
health_r_a, health_p =stats.pearsonr(df_combined['PARK_ACRES'], df_combined['Value'])

acres = plt.scatter(df_combined['PARK_ACRES'], df_combined['Value'])
plt.ylabel('Dollars of Health costs per acre')
plt.xlabel('Acres of park')
plt.title('Health Cost Scatter')
print("health r, num parks:")
print(health_r_n)
print("health r, park acres:")
print(health_r_a)
#column times correlation, last column add everything up 
health_df = df_combined

<img src="Pr_045-master/health.png">

### Housing
The housing data we had contains a few randomly sampled house values for each beat. We first grouped these values by beat and then averaged the values to get the average house value per beat. Because we got the average house value per beat we did not need to normalize the values by beat size. There is a positive correlation value, showing that more parks correlate to higher house prices.

In [ ]:
df_housing = pd.read_csv('data/housing/finalHouseValuation.csv')
df_housing.drop('clairemont mesa east', axis=1, inplace=True)
df_housing=df_housing.rename(columns = {'495085':'Value'})
df_housing=df_housing.rename(columns = {'111':'Beat'})

# Average over samples per beat
df_housing = df_housing.groupby('Beat').mean()
df_housing['Beat'] = df_housing.index

df_combined = pd.merge(df_parks, df_housing, on='Beat', how='left')
df_combined = df_combined.fillna(value=np.mean(df_combined['Value']))

housing_r_n, housing_p =stats.pearsonr(df_combined['NUM_OF_PARKS'], df_combined['Value'])
housing_r_a, housing_p =stats.pearsonr(df_combined['PARK_ACRES'], df_combined['Value'])
acres = plt.scatter(df_combined['PARK_ACRES'], df_combined['Value'])
print("housing r, num:")
print(housing_r_n)
print("housing r, acres:")
print(housing_r_a)

plt.ylabel('Average housing value for beat')
plt.xlabel('Acres of park')
plt.title('House Value Scatter')

<img src="Pr_045-master/house.png">

### Police Dispatch
For police dispatches we first negated and then normalized all of our values for police dispatch cost by the beat size. This is because dispatches cost the beat money, and larger beats will generally have more police dispatches. When we ran our regression we realized that the police data we collected contained a few outliers of beasts with very high ratios of police dispatches to area. We decided to remove these outliers in our final regression. There is a slightly positive correlation value, showing that more parks correlate to a smaller ratio of police dispatches/area in a beat.

In [ ]:
df_police_dispatch = pd.read_csv('data/police_dispatch/police_dispatch_dollars_beat.csv')
df_police_dispatch.drop('Unnamed: 0', axis = 1, inplace = True)
df_police_dispatch.drop('index', axis = 1, inplace = True)
df_police_dispatch=df_police_dispatch.rename(columns = {'dollars':'Value'})
df_police_dispatch=df_police_dispatch.rename(columns = {'beat_n':'Beat'})
df_police_dispatch['Value']=df_police_dispatch['Value'].map(lambda x: -x)

df_combined = pd.merge(df_parks, df_police_dispatch, on='Beat', how='left')
df_combined = df_combined.fillna(value=0)

for i in range(0, len(df_combined)):
    df_combined.ix[i, 'Value'] =  df_combined.ix[i, 'Value']/ df_combined.ix[i, 'BEAT_AREA']

# Remove outliers
df_combined = rem_outlier(df_combined, 'Value', 3, how='min')

pd_r_n, pd_p =stats.pearsonr(df_combined['NUM_OF_PARKS'], df_combined['Value'])
pd_r_a, pd_p =stats.pearsonr(df_combined['PARK_ACRES'], df_combined['Value'])
acres = plt.scatter(df_combined['PARK_ACRES'], df_combined['Value'])
print("pd r, num:")
print(pd_r_n)
print("pd r, acres:")
print(pd_r_a)
police_df = df_combined

plt.ylabel('Police dispatch cost per acre')
plt.xlabel('Acres of park')
plt.title('Police Dispatch Scatter')

<img src="Pr_045-master/police.png">

### Recreational Activities
For recreational activities, we were not able to get data for each beat. We assumed that if we did not have a value for a beat, there was no added value to the beat. Thus we gave beats with no value a value of 0. We then ran our regression after removing outliers and saw a strong positive correlation which is what we were expecting. 

In [ ]:
df_community = pd.read_csv('data/community/activities_beats_money.csv')
df_community.drop('Unnamed: 0', axis = 1, inplace = True)
df_community=df_community.rename(columns = {'Activity_price':'Value'})

beat = df_community.iloc[0]['Beat']
beat

df_community['Beat'] = df_community['Beat'].map(lambda x: x[1:4])
df_community = df_community[df_community['Beat'] != '-1,']


df_community['Beat'] = df_community['Beat'].astype(int)

df_combined = pd.merge(df_parks, df_community, on='Beat', how='left')
df_combined = df_combined.fillna(value=0)

# Remove outliers
df_combined = rem_outlier(df_combined, 'Value', 1, how='max')

comm_r_n, comm_p =stats.pearsonr(df_combined['NUM_OF_PARKS'], df_combined['Value'])
comm_r_a, comm_p =stats.pearsonr(df_combined['PARK_ACRES'], df_combined['Value'])
acres = plt.scatter(df_combined['PARK_ACRES'], df_combined['Value'])
print("comm r, num:")
print(comm_r_n)
print("comm r, acres:")
print(comm_r_a)
community_df = df_combined

plt.ylabel('Value derived from community activies')
plt.xlabel('Acres of park')
plt.title('Community Activities Scatter')

<img src="Pr_045-master/community.png">

### Storm Drains
The environment data we had revolved around storm drains, with the reasoning that parks could serve as a natural runoff for storm water and reduce the cost of building storm drains. We were able to get the locations of the storm drains and group them by beats. After counting the total number of storm drains in a beat, we normalized the data by beat area to account for the disparity in beat area sizes. We then correlated it with park area per beat and were able to see a positive correlation

In [ ]:
df_env = pd.read_csv('data/Environment/env_value.csv')

df_env.drop('Count', axis = 1, inplace=True)
df_env.drop('Name', axis = 1, inplace = True)
df_env.drop('Unnamed: 0', axis = 1, inplace = True)

df_combined = pd.merge(df_parks, df_env, on='Beat', how='left')
df_combined = df_combined.fillna(value=0)

for i in range(0, len(df_combined)):
    df_combined.ix[i, 'Value'] =  df_combined.ix[i, 'Value']/ df_combined.ix[i, 'BEAT_AREA']


df_combined = rem_outlier(df_combined, 'Value', 3, how='min')
    
env_r_n, env_p =stats.pearsonr(df_combined['NUM_OF_PARKS'], df_combined['Value'])
env_r_a, env_p =stats.pearsonr(df_combined['PARK_ACRES'], df_combined['Value'])
plt.scatter(df_combined['PARK_ACRES'], df_combined['Value'])
print("env r, num:")
print(env_r_n)
print("env r, acres:")
print(env_r_a)
env_df = df_combined

plt.ylabel('Cost of storm drains per acre')
plt.xlabel('Acres of park')
plt.title('Storm Drain Scatter')

<img src="Pr_045-master/env_hm.png">

### Park Models


In [ ]:
AIR_VAL_PER_ACRE = 1000
weights = {'health': health_r_a, 'housing': housing_r_a, 'police': pd_r_a,'community': comm_r_a, 'environment': env_r_a, 'air': AIR_VAL_PER_ACRE}
dfs = {'health': health_df, 'housing': housing_df, 'police': police_df, 'community': community_df, 'environment': env_df, 'air': df_parks['PARK_ACRES']}

variables = ['health', 'housing', 'police', 'community','environment',]
equations = ['air',]

avg_beat_area = np.mean(dfs[variables[0]]['BEAT_AREA'])
beets = dfs[variables[0]]['Beat']
nums  = dfs[variables[0]]['NUM_OF_PARKS']
acres = dfs[variables[0]]['PARK_ACRES']

# Multiply env and police by average beat size to get dollars back
weights['police'] *= avg_beat_area
weights['environment'] *= avg_beat_area

# Center values around zero, so that outstanding value is provided, not bias
# ($1m house is worth only slightly more because of location)
for data in variables:
    dfs[data] = dfs[data]['Value'] - np.mean(dfs[data]['Value'])
    
first_one = variables[0]
dfp = dfs[first_one]*weights[first_one]
for var in variables[1:]:
    dfp += dfs[var]*weights[var]
for eqn in equations:
    dfp += dfs[eqn]*weights[eqn]
    
# Add back in beat numbers, nums, acres
dfp = pd.DataFrame(dfp)
dfp['Beat'] = beets
dfp['Num'] = nums
dfp['Acres'] = acres

# Value of parks in each beat, positive means parks add a lot of value, negative means they need more parks
gen_heatmap(dfp, "RdYlGn")

The map below indicates which beats are gaining a lot of total value from their parks. The parks that are red could use more parks. 
<img src="Pr_045-master/heat1.png">

In [ ]:
# Prep value of parks per num in beats with parks

dfp = dfp.clip(lower=0)
dfp = dfp.fillna(value=0)

dfp = dfp.clip(lower=0)

dfp = dfp[dfp.Num != 0]
dfpn = dfp
dfpn['Value'] = dfp['Value']/dfp['Num']
gen_heatmap(dfpn, "Greens")

This map shows the average value each individual park is providing to the community for each beat. The darker green beats have parks that provide a lot of value for each park
<img src="Pr_045-master/heat2.png">

## Results
---

After conducting our analysis, we found that many of the features we decided to explore did indeed have a correlation with the presence of public parks. 

Using Pearsons correlation, we found that the amount of money spent on health care costs and the total acreage of park space has an r value of .2818. This indicates a positive relationship of communities with lower health care costs containing larger spaces for public parks. 

Similarly, the correlation statistic between housing and park acreage is .1709. Although we predicted that housing prices would have a stronger correlation with park acreage, we found that our reason for error was that we werent able to scrape enough data from Zillow in order to produce a dataset that was fully representative of all police beats. 

The relationship between police activity and park acreage also had an r value of .2181, and the correlation of storm drains to park acreage was .1155. We were surprised to find that the strongest correlation of features was from the community centers and recreational facilities, our analysis produced an r value of .7266. 

After conducting the individual analysis for each feature, we generated two final heat maps: 

1) The first heat map shows the total value these features have had on a particular police beat and the positive impact the parks have had. The regions marked with darker, redder shades indicate that the establishment of parks in these areas would provide the greatest value and are necessary. 

2) The second, and final, heat map that we generated shows how much value a police beat has gained from a single public park. It is important to note that some of the regions with the greatest value gained from an individual park are centered around downtown San Diego, where parks are typically most accessible and fully utilized by the surrounding community. 


## Conclusion
---
As our hypothesis suggested, we have found that the existence of different features within a community seems to have a positive influence on public parks. Our analysis not only shows information about current parks, but it also predicts the success and necessity of future parks. This allows city planners to consider increasing the public space allocated for parks in the specific regions we have indicated above. 

How can we improve our analysis of the value of public parks? 

By increasing the size of our datasets and collecting more observations per feature, we can increase the certainty and confidence in our results. Even though the datasets we collected each covered a different feature, our analysis may have still been limited. Our initial decision to focus on these five features to evaluate the success of public parks may have been biased and it is possible that there are many other contributing factors (such as school location and public transportation) that positively influence the success and necessity of parks. However, we have best accounted for these oversights in our initial analysis by focusing on collecting complete and adequate data for the few features that we chose and using the data to produce comprehensive analyses and results. 

